In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_experimental.text_splitter import SemanticChunker
import chromadb
import ollama
import os
from tqdm import tqdm
from langchain_chroma import Chroma
from langchain.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain.chains import RetrievalQA
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate

from langchain_text_splitters import RecursiveCharacterTextSplitter

In [4]:
path=r"D:/HRAgent/Policies/ANTI-BRIBERY AND ANTI-CORRUPTION POLICY OF TATA INDUSTRIES.pdf"

In [5]:
loader=PyPDFLoader(path)

In [6]:
resultantpdf=[]

In [7]:
resultantpdf.extend(loader.load())

In [8]:
url="http://172.16.0.178:8010"

In [11]:
embedding=OllamaEmbeddings(model="mxbai-embed-large",base_url=url)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
                                                chunk_size=250,
                                                chunk_overlap=50,
                                                length_function=len,
                                                is_separator_regex=False,
                                            )
print("=======chunk creation=========") 
chunks= text_splitter.create_documents([doc.page_content for doc in tqdm(resultantpdf)])

In [ ]:
chroma_db = Chroma.from_documents(
    documents=chunks, 
    embedding=embedding, 
    persist_directory="knowledgebase", 
    collection_name="recursive_embed_knolwdge"
)

In [ ]:
chroma_db.as_retriever().get_relevant_documents("finance vice persident of tata insights and quants")

In [2]:
prompt_template = """Use the following pieces of context to answer the question at the end. 
If you do not know the answer, please think rationally and answer from your own knowledge base.
Don't put extra information

{context}

Question: {question}
"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [9]:
llm=ChatOllama(base_url=url,model="llama3.2:1b",temperature=0.1)

In [19]:
retriever=Chroma(
            collection_name="knowledgebase",
            persist_directory="recursive_embed_knolwdgev1",
            embedding_function=embedding

        ).as_retriever()

In [20]:
query="tell me about tata iq?"

In [21]:
chain_type_kwargs = {"prompt": PROMPT}
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff", 
    retriever=retriever, 
    chain_type_kwargs=chain_type_kwargs)

qa_chain.run(query)

"Tata Iq is a mobile phone brand owned by Tata Consultancy Services (TCS), an Indian multinational technology company. It was launched in 2007 as the second mobile phone brand after Nokia, with the goal of competing with Apple's iPhone and other high-end smartphones.\n\nThe first generation of Tata Iq phones featured Android operating system and were available in various storage capacities, including 8GB and 16GB. The phones had a range of features, including Wi-Fi connectivity, Bluetooth, and GPS.\n\nTata Iq was discontinued by TCS in 2012 due to poor sales performance."

NameError: name 'BaseModel' is not defined